In [30]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from annoq_tree_gen import DTYPES
from base import load_json

annotations = pd.read_csv("./../../annoq-data/revised_tree.csv", sep=',', dtype=DTYPES)
terms = pd.read_csv("./../../annoq-data/slim/terms.tsv", sep='\t')
mappings = load_json("./../../annoq-data/mappings.json")

annotations.set_index("id", inplace=True, drop=False)

term_cols = {
    'ID':'id',
    'LABEL':'label',
    'SubClass Of':'parents',
    'hasOBONamespace':'aspect'
}

terms.rename(columns=term_cols, inplace=True)
terms.set_index("id", inplace=True, drop=False)

In [31]:
def is_leaf(G, node):
    return G.out_degree(node) == 0 and G.in_degree(node) == 1

def add_value_type(name):
    if 'GO_' in name and 'list_id' in name:
        return 'GO_id'
    if 'GO_' in name and 'list' in name:
        return 'GO_label'
        
    return None


annotations['value_type'] = annotations.apply(lambda row: add_value_type(row['name']),axis=1)
annotations

,id,parent_id,leaf,name,label,detail,link,pmid,sort,value_type,count,children_count,category_count,leaf_count,root URL,sample URL
id,,,,,,,,,,,,,,,,
0,0,NaN,False,root,Annotation,NaN,NaN,NaN,0.0,None,607,625,18,607,NaN,NaN
1,1,0,False,Basic Info,NaN,"Basic information about the variant, such as c...",NaN,NaN,1.0,None,5,5,0,5,NaN,NaN
26,26,0,False,ANNOVAR,NaN,Pre-computed ANNOVAR annotations for all alter...,http://annovar.openbioinformatics.org/en/lates...,20601685,2.0,None,56,56,0,56,NaN,NaN
208,208,0,False,SnpEff,NaN,AnpEff is a program for annotating and predict...,http://pcingola.github.io/SnpEff/,22728672,3.0,None,79,79,0,79,NaN,NaN
132,132,0,False,VEP,NaN,Variant Effect Predictor (VEP) is developed by...,https://uswest.ensembl.org/info/docs/tools/vep...,27268795,4.0,None,75,75,0,75,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,495,True,sno_miRNA_type,NaN,the type of snoRNA or miRNA (from miRBase/snoR...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN
622,622,495,True,splicing_consensus_ada_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN
623,623,495,True,splicing_consensus_rf_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN


In [32]:
g = nx.from_pandas_edgelist(
    annotations, 'parent_id', 'id', create_using=nx.DiGraph())

roots = [annotations.loc[n]['name'] for n in nx.descendants(g,'26') if is_leaf(g, n)]

#len(roots)

def leaf_count(G, source):
    children = [n for n in nx.descendants(G, source) if is_leaf(g, n)]
    return len(children)


def descendants_count(G, source):
    children = [n for n in nx.descendants(G, source)]
    return len(children)

def parents_count(G, source):
    children = [n for n in nx.descendants(G, source) if not is_leaf(g, n)]
    return len(children)

def get_type(mapping_dict, field):
    try:
        field_key =  mapping_dict["annoq-test"]["mappings"]["properties"][field]

        if field_key != None:
            return field_key['type']
    
    except KeyError:
        return np.nan
        
    
get_type(mappings, 'ref')
#child_count(g, '26')
#nx.descendants(g,'26')

'text'

In [33]:
annotations['children_count'] = annotations.apply(lambda row: descendants_count(g, row['id']),axis=1)
annotations['category_count'] = annotations.apply(lambda row: parents_count(g, row['id']),axis=1)
annotations['leaf_count'] = annotations.apply(lambda row: leaf_count(g, row['id']),axis=1)
annotations['field_type'] = annotations.apply(lambda row: get_type(mappings, row['name']),axis=1)
annotations

,id,parent_id,leaf,name,label,detail,link,pmid,sort,value_type,count,children_count,category_count,leaf_count,root URL,sample URL,field_type
id,,,,,,,,,,,,,,,,,
0,0,NaN,False,root,Annotation,NaN,NaN,NaN,0.0,None,607,625,18,607,NaN,NaN,NaN
1,1,0,False,Basic Info,NaN,"Basic information about the variant, such as c...",NaN,NaN,1.0,None,5,5,0,5,NaN,NaN,NaN
26,26,0,False,ANNOVAR,NaN,Pre-computed ANNOVAR annotations for all alter...,http://annovar.openbioinformatics.org/en/lates...,20601685,2.0,None,56,56,0,56,NaN,NaN,NaN
208,208,0,False,SnpEff,NaN,AnpEff is a program for annotating and predict...,http://pcingola.github.io/SnpEff/,22728672,3.0,None,79,79,0,79,NaN,NaN,NaN
132,132,0,False,VEP,NaN,Variant Effect Predictor (VEP) is developed by...,https://uswest.ensembl.org/info/docs/tools/vep...,27268795,4.0,None,75,75,0,75,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,621,495,True,sno_miRNA_type,NaN,the type of snoRNA or miRNA (from miRBase/snoR...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN,text
622,622,495,True,splicing_consensus_ada_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN,float
623,623,495,True,splicing_consensus_rf_score,NaN,splicing-change prediction for splicing consen...,NaN,NaN,NaN,None,0,0,0,0,NaN,NaN,float


In [35]:
annotations.to_csv('./../../annoq-data/tree-with-types.csv', index = False)